In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('fake_job_postings.csv')
df.dropna()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
6,7,Head of Content (m/f),"DE, BE, Berlin",ANDROIDPIT,20000-28000,"Founded in 2009, the Fonpit AG rose with its i...",Your Responsibilities: Manage the English-spea...,Your Know-How: ...,Your Benefits: Being part of a fast-growing co...,0,1,1,Full-time,Mid-Senior level,Master's Degree,Online Media,Management,0
15,16,VP of Sales - Vault Dragon,"SG, 01, Singapore",Sales,120000-150000,Jungle Ventures is the leading Singapore based...,About Vault Dragon Vault Dragon is Dropbox for...,Key Superpowers3-5 years of high-pressure sale...,"Basic: SGD 120,000Equity negotiable for a rock...",0,1,1,Full-time,Executive,Bachelor's Degree,Facilities Services,Sales,0
23,24,"Vice President, Sales and Sponsorship (Busines...","US, CA, Carlsbad",Businessfriend.com,100000-120000,"WDM Group is an innovative, forward thinking d...",#URL_eda2500ddcedb60957fcd7f5b164e092966f8c4e8...,"Job Requirements:A reputation as a ""go-getter""...",Businessfriend will offer a competitive six fi...,0,1,0,Full-time,Executive,Unspecified,Internet,Sales,0
98,99,IC&E Technician,"US, , Stocton, CA",Oil & Energy,95000-115000,...,"IC&amp;E Technician | Bakersfield, CA Mt. Poso...","QualificationsKnowledge, Skills &amp; Abilitie...",BENEFITSWhat is offered:Competitive compensati...,0,1,1,Full-time,Mid-Senior level,High School or equivalent,Oil & Energy,Other,1
102,103,Marketing Administrator,"GB, WAR, Coventry",Marketplace,15000-18000,Renewable Energy and Environmental Protection ...,The job is to support the growth of the #URL_9...,"Computer literateAble to work with HTML, altho...",For a suitably motivated and success orientate...,1,1,0,Full-time,Entry level,Bachelor's Degree,Internet,Marketing,0
134,135,Senior Business Development Manager,"GB, , Manchester",Sales and Business Development,70000-90000,MarketInvoice is one of the most high-profile ...,BACKGROUNDA senior sales and account managemen...,5-10 years sales experience in B2B solution sa...,Based in our Manchester officeCompetitive basi...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Financial Services,Sales,0
153,154,Senior Project Manager,"GB, , London",Production,28000-45000,"Neverbland is a team of designers, developers ...",Some of us are born with the ability to multi ...,2-3 years agency experience in a Producer/Proj...,"Work in a dynamic, creative environment.Learn ...",0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Internet,Production,0
160,161,Field Services Supervisor,"US, WI, Western Wisconsin",Field Operations,44000-57000,"Dairy, food and beverage processors work withi...","W.M. Sprinkman designs, builds and installs st...",5-7 years of experience in fabrication and/or ...,We provide a solid benefits package with healt...,0,0,0,Full-time,Associate,High School or equivalent,Mechanical or Industrial Engineering,Manufacturing,0
179,180,Internal Recruiter,"GB, LND, London",Playfair Capital,20000-40000,Playfair Capital is an early stage technology ...,You will be responsible for hiring across a po...,Previous recruitment experience in fast growth...,Work will an awesome group of companies and be...,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Information Technology and Services,Human Resources,0
197,198,Software developer,"IN, MP, Indore",Software development,360000-600000,"Being into the IT domain for over 4 years, Wal...","Is coding just a task for you, or something to...",Min. 1 year in Web Development,In-house product development with a chance to ...,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Information Technology and Services,Information Technology,0


In [3]:
len(df[df.fraudulent==0])

17014

In [4]:
#upsampling minority class
from sklearn.utils import resample

df_majority = df[df.fraudulent==0]
df_minority = df[df.fraudulent==1]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     
                                 n_samples=17014,   
                                 random_state=123)

df_upsampled = pd.concat([df_majority, df_minority_upsampled])

Naive Baiye's Classifier

In [5]:
sub1 = df_upsampled[['telecommuting','fraudulent']]

In [6]:
import sklearn

In [7]:
from sklearn.naive_bayes import BernoulliNB,GaussianNB
from sklearn.model_selection import train_test_split
import numpy as np

In [8]:
X = sub1['telecommuting']
y = sub1['fraudulent']

In [9]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.8,stratify=y)

In [10]:
clf = GaussianNB()

In [11]:
clf.fit(np.array(X_train).reshape(-1, 1),y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [12]:
y_pred = clf.predict(np.array(X_test).reshape(-1, 1))

In [13]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,classification_report

In [14]:
f"accracy_Score is {accuracy_score(y_test,y_pred)}"

'accracy_Score is 0.5187525254380487'

In [15]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.51      0.96      0.67     13612
           1       0.65      0.08      0.14     13611

   micro avg       0.52      0.52      0.52     27223
   macro avg       0.58      0.52      0.40     27223
weighted avg       0.58      0.52      0.40     27223



Adding two additional features 'has_Company_logo' and 'has questions'

In [16]:
sub2 = df_upsampled[['telecommuting','has_company_logo','has_questions','fraudulent']].dropna()

In [17]:
X = sub2[['telecommuting','has_company_logo','has_questions']]
y = sub2['fraudulent']

In [18]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.8,stratify=y)

Trying Random forest and gradientboosting classifier

In [19]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [20]:
def print_results(results):
    print('BEST PARAMS: {}\n'.format(results.best_params_))

    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))

In [21]:
rf = RandomForestClassifier()
parameters = {
    'n_estimators': [5, 50, 250],
    'max_depth': [2, 4, 8, 16, 32, None]
}

cv = GridSearchCV(rf, parameters, cv=5)
cv.fit(X_train, y_train.values.ravel())

print_results(cv)

BEST PARAMS: {'max_depth': 2, 'n_estimators': 50}

0.752 (+/-0.024) for {'max_depth': 2, 'n_estimators': 5}
0.755 (+/-0.023) for {'max_depth': 2, 'n_estimators': 50}
0.754 (+/-0.02) for {'max_depth': 2, 'n_estimators': 250}
0.755 (+/-0.023) for {'max_depth': 4, 'n_estimators': 5}
0.755 (+/-0.023) for {'max_depth': 4, 'n_estimators': 50}
0.755 (+/-0.023) for {'max_depth': 4, 'n_estimators': 250}
0.755 (+/-0.023) for {'max_depth': 8, 'n_estimators': 5}
0.755 (+/-0.023) for {'max_depth': 8, 'n_estimators': 50}
0.755 (+/-0.023) for {'max_depth': 8, 'n_estimators': 250}
0.755 (+/-0.023) for {'max_depth': 16, 'n_estimators': 5}
0.755 (+/-0.023) for {'max_depth': 16, 'n_estimators': 50}
0.755 (+/-0.023) for {'max_depth': 16, 'n_estimators': 250}
0.755 (+/-0.023) for {'max_depth': 32, 'n_estimators': 5}
0.755 (+/-0.023) for {'max_depth': 32, 'n_estimators': 50}
0.755 (+/-0.023) for {'max_depth': 32, 'n_estimators': 250}
0.755 (+/-0.023) for {'max_depth': None, 'n_estimators': 5}
0.755 (+/-0.02

GradientBoostingClassifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gb = GradientBoostingClassifier()
parameters = {
    'n_estimators': [5, 50, 250, 500],
    'max_depth': [1, 3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 1, 10, 100]
}

cv = GridSearchCV(gb, parameters, cv=5)
cv.fit(X_train, y_train.values.ravel())

print_results(cv)

In [ ]:
y_pred = cv.predict(np.array(X_test))

In [ ]:
f"accracy_Score is {accuracy_score(y_test,y_pred)}"

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
#Multilayerperceptron(Neuralnetwork classifier)
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier()
parameters = {
    'hidden_layer_sizes': [(10,), (50,), (100,)],
    'activation': ['relu', 'tanh', 'logistic'],
    'learning_rate': ['constant', 'invscaling', 'adaptive']
}

cv = GridSearchCV(mlp, parameters, cv=5)
cv.fit(X_train, y_train.values.ravel())

print_results(cv)

In [ ]:
df_upsampled[df_upsampled.fraudulent == 1]

Classifier with description text feauture

In [22]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
stop_words = list(stopwords.words('english')) 

In [23]:
vectorizer = TfidfVectorizer(max_features=30,stop_words=set(stop_words),use_idf=True, smooth_idf=True,ngram_range = (1,1))
sub3 = df_upsampled[['description','fraudulent']].dropna()
X = vectorizer.fit_transform(sub3['description'])
features = (vectorizer.get_feature_names()) 
y = sub3['fraudulent']

In [24]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.8,stratify=y)

In [25]:
#Multilayerperceptron(Neuralnetwork classifier)
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier()
parameters = {
    'hidden_layer_sizes': [(100,)],
    'activation': ['relu'],
    'learning_rate': ['constant']
}

cv = GridSearchCV(mlp, parameters, cv=5)
cv.fit(X_train, y_train.values.ravel())

print_results(cv)

C:\Users\Raviraj\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Raviraj\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Raviraj\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Raviraj\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, 

C:\Users\Raviraj\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Raviraj\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Raviraj\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Raviraj\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, 

C:\Users\Raviraj\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Raviraj\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Raviraj\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Raviraj\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, 

BEST PARAMS: {'activation': 'relu', 'hidden_layer_sizes': (100,), 'learning_rate': 'constant'}

0.774 (+/-0.012) for {'activation': 'relu', 'hidden_layer_sizes': (10,), 'learning_rate': 'constant'}
0.775 (+/-0.023) for {'activation': 'relu', 'hidden_layer_sizes': (10,), 'learning_rate': 'invscaling'}
0.77 (+/-0.029) for {'activation': 'relu', 'hidden_layer_sizes': (10,), 'learning_rate': 'adaptive'}
0.866 (+/-0.019) for {'activation': 'relu', 'hidden_layer_sizes': (50,), 'learning_rate': 'constant'}
0.857 (+/-0.013) for {'activation': 'relu', 'hidden_layer_sizes': (50,), 'learning_rate': 'invscaling'}
0.858 (+/-0.014) for {'activation': 'relu', 'hidden_layer_sizes': (50,), 'learning_rate': 'adaptive'}
0.893 (+/-0.014) for {'activation': 'relu', 'hidden_layer_sizes': (100,), 'learning_rate': 'constant'}
0.889 (+/-0.02) for {'activation': 'relu', 'hidden_layer_sizes': (100,), 'learning_rate': 'invscaling'}
0.89 (+/-0.01) for {'activation': 'relu', 'hidden_layer_sizes': (100,), 'learning_

C:\Users\Raviraj\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [28]:
y_pred = cv.predict(X_test)

In [30]:
accuracy_score(y_test,y_pred)

0.8893421536199927

In [31]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.90      0.87      0.89     13611
           1       0.88      0.90      0.89     13599

   micro avg       0.89      0.89      0.89     27210
   macro avg       0.89      0.89      0.89     27210
weighted avg       0.89      0.89      0.89     27210



In [32]:
df_upsampled

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0
5,6,Accounting Clerk,"US, MD,",NaN,NaN,NaN,Job OverviewApex is an environmental consultin...,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,0
6,7,Head of Content (m/f),"DE, BE, Berlin",ANDROIDPIT,20000-28000,"Founded in 2009, the Fonpit AG rose with its i...",Your Responsibilities: Manage the English-spea...,Your Know-How: ...,Your Benefits: Being part of a fast-growing co...,0,1,1,Full-time,Mid-Senior level,Master's Degree,Online Media,Management,0
7,8,Lead Guest Service Specialist,"US, CA, San Francisco",NaN,NaN,Airenvy’s mission is to provide lucrative yet ...,Who is Airenvy?Hey there! We are seasoned entr...,"Experience with CRM software, live chat, and p...",Competitive Pay. You'll be able to eat steak e...,0,1,1,NaN,NaN,NaN,NaN,NaN,0
8,9,HP BSM SME,"US, FL, Pensacola",NaN,NaN,Solutions3 is a woman-owned small business who...,Implementation/Configuration/Testing/Training ...,MUST BE A US CITIZEN.An active TS/SCI clearanc...,NaN,0,1,1,Full-time,Associate,NaN,Information Technology and Services,NaN,0
9,10,Customer Service Associate - Part Time,"US, AZ, Phoenix",NaN,NaN,"Novitex Enterprise Solutions, formerly Pitney ...",The Customer Service Associate will be based i...,Minimum Requirements:Minimum of 6 months custo...,NaN,0,1,0,Part-time,Entry level,High School or equivalent,Financial Services,Customer Service,0
